Задание 1. (5 баллов) 
В тетрадке реализована биграмная языковая модель (при генерации учитывается информация только о 1 предыдущем слове). Реализуйте триграмную модель и сгенерируйте несколько текстов. Сравните их с текстами, сгенерированными биграмной моделью. 
Можно использовать те же тексты, что в семинаре, или взять какой-то другой (на английском или русском языке).  

Делать это задание будет легче после прочтения первых 7 страниц вот этой главы из Журафского - https://web.stanford.edu/~jurafsky/slp3/3.pdf



In [229]:
import nltk
import re
from collections import defaultdict
import copy
from gensim.models.phrases import Phrases

In [230]:
with open('stranger.txt', encoding='utf-8') as file: #Robert Heinlein's Stranger in a Strange Land with preface removed
    stranger = file.read()

In [231]:
stranger = stranger.replace('“Stranger In A Strange Land” by Robert Heinlein', '')

In [232]:
sents = nltk.tokenize.sent_tokenize(stranger)

*Не включаю слова, написанные через дефис, т.к. 1) в данном тексте не используются тире 2) между дефисами нет пробелов.*

In [233]:
pattern = re.compile(r'([A-Za-z]+[\']?[A-Za-z]*)')

In [234]:
sents = [re.findall(pattern, sent) for sent in sents]

In [235]:
tri_model = defaultdict(lambda: defaultdict(lambda: 0))
for sentence in sents:
    for w1, w2, w3 in nltk.trigrams(sentence, pad_right=True, pad_left=True, left_pad_symbol='<s>', right_pad_symbol='</s>'):
        tri_model[(w1, w2)][w3] += 1

Логарифм не использую, потому что верятности нужны только, чтобы сделать взвешенную выборку.

In [236]:
for bigram in tri_model:
    total_count = sum(tri_model[bigram].values())
    for target in tri_model[bigram]:
        tri_model[bigram][target] /= total_count

In [237]:
def tri_generate(model, start=('<s>', '<s>')):
    text = list(start)
    while text[-1] != '</s>': 
        index = tuple(text[-2:])
        keys = list(model[index].keys())
        values = list(model[index].values())
        key = np.random.choice(keys, 1, values)[0]
        text.append(key)
    return ' '.join(text[2:]).strip(' </s>')

In [238]:
def text_generator(sent_generator, model, number_of_sents=1, count_words=False):
    result = []
    for _ in range(number_of_sents):
        result.append(sent_generator(model))
    if count_words == True:
        count = count_words_avg(result)
        return count
    else:
        result = '. '.join(result) + '.'
        return result


In [239]:
def count_words_avg(sents):
    total = 0
    for sent in sents:
        total += len(sent.split(' '))
    return total/len(sents)

*Тут надо отметить, что данный текст – результат работы программы автоматического распознавания печатной книги, поэтому здесь нередко встречаются ошибки. Их можно было бы поправить, но это в масштаб данного задания всё же не входит.*

In [249]:
tri_example = text_generator(tri_generate, tri_model, 6)
tri_example

"Supreme Bishop and you won't believe it was still climbing. Lord love you it's a short delay from the Secretary won't come in through here. Finally Jubal said I'll stick to stairs and to let the infant with both of genius level. Moreover he could figure a deal that might balk them. Fly. Stinky taught it in Mike many times each day."

In [250]:
bi_model = defaultdict(lambda: defaultdict(lambda: 0))
for sentence in sents:
    for w1, w2 in nltk.bigrams(sentence, pad_right=True, pad_left=True, left_pad_symbol='<s>', right_pad_symbol='</s>'):
        bi_model[w1][w2] += 1

In [251]:
for unigram in bi_model:
    total_count = sum(bi_model[unigram].values())
    for target in bi_model[unigram]:
        bi_model[unigram][target] /= total_count

In [252]:
def bi_generate(model, start=['<s>']):
    text = copy.copy(start)
    while text[-1] != '</s>': 
        index = text[-1]
        keys = list(model[index].keys())
        values = list(model[index].values())
        key = np.random.choice(keys, 1, values)[0]
        text.append(key)
    return ' '.join(text[1:]).strip(' </s>')

In [254]:
bi_example = text_generator(bi_generate, bi_model, 6)
bi_example

"Nevertheless they supported their priest someday and anned guard placed herself he manages God didn't speak wrongly at Jubal's favorite children Kids in dressed up of Douglas concluded still places half century of truth interwoven with weak spot in heat lightning bug. VI had his aid kits in how darling I'm slow motion monestary garden area around them their weapons disappear one longish pause. Kiss quite incapable of grace of fourthrate prophet who hold even removed his freezer if Captain and went there. Beginners' classes for town suddenly reminded himself tightly when to alcoholic brotherhood. Martinis Think it there can save you catch his major skills although I ride in JubaL he protested shrilly Where's my forties whom Harshaw reached the dictionary which often needs your religion claims her against earlier hour ago just say wearing was already lived as hard part II before even talk Jubal knowing nothing to evade it bliss because it's that s three each differently. Can't say somet

Результат триграммной модели для сравнения: <br> <br>

In [256]:
tri_example

"Supreme Bishop and you won't believe it was still climbing. Lord love you it's a short delay from the Secretary won't come in through here. Finally Jubal said I'll stick to stairs and to let the infant with both of genius level. Moreover he could figure a deal that might balk them. Fly. Stinky taught it in Mike many times each day."

Можно отметить две вещи: <br>
1) Предложения, созданные триграммной модели определённо больше похожи на текст, написанный человеком. Они более связные (только грамматически, естественно). <br>
2) Биграммные предложения длиннее, чем триграммные, при условии, что мы останавливаемся только на символе окончания предложения. Это подтверждается экспериментом ниже.

In [257]:
n = 100
m = 10
bi_test = 0
tri_test = 0
for _ in range(n):
    bi_test += text_generator(bi_generate, bi_model, m, count_words=True)
    tri_test += text_generator(tri_generate, tri_model, m, count_words=True)
print(f'Average bigram model sentence length: {bi_test/n:.2f} words\n' +
      f'Average trigram model sentence length: {tri_test/n:.2f} words\n')

Average bigram model sentence length: 19.59 words
Average trigram model sentence length: 12.00 words



Задание 2. (5 баллов) 
При помощи gensim.models.Phrases реализуйте byte-pair-encoding, про который говорилось на первом семинаре (https://github.com/mannefedov/compling_nlp_hse_course/blob/master/notebooks/Preprocessing.ipynb) 
А именно 1) возьмите любой текст; разбейте его на предложения, а каждое предложение разбейте на отдельные символы (не потеряйте пробелы) 2) обучите gensim.models.Phrases на полученных символьных предложениях 3) примените полученный нграммер к этим символьным предложениям 4) повторите 2 и 3 N количество раз, чтобы начали получаться целые слова
Параметры в gensim.models.Phrases влияют на количество получаемых нграммов после каждого прохода, поэтому не забудьте их настроить


In [262]:
symbol_sents = [' '.join(sent) for sent in sents]

In [263]:
symbol_sents = [[ch for ch in sent if ch not in ',.;!?\n'] for sent in symbol_sents]

In [264]:
def symbol_grams(sents, iterations):
    transformed = []
    for _ in range(iterations):
        if not transformed:
            transformed = sents
        phrases = Phrases(transformed, scoring='npmi', threshold=0, min_count=2)
        transformed = phrases[transformed]
    return transformed

In [265]:
result = symbol_grams(symbol_sents, 3)

Как видим, действительно, используя такой метод, не получается получить нграммы соответствующие именно отдельным словам, не перепрыгивая через пробелы.

In [266]:
list(result)[1] 

['V_a_l_e_n_t_i_n',
 'e_ _M_i_c_h_a',
 'e_l_ _S_m_i_t_h',
 ' _w_a_s_ _a_s_ ',
 'r_e_a_l_ _a_s_ ',
 't_a_x_e_s_ _b',
 'u_t_ _h_e_ _w_a',
 's_ _a_ _r',
 'a_c_e_ _o_f_ ',
 'o_n_e']

In [267]:
def split_spaces(sents):
    res = []
    for sent in sents:
        sub = []
        for word in sent:
            sub.extend(word.split(' '))
        sub = [word.strip('_')  if word else ' ' for word in sub] 
        res.append(sub)   
    return res    

In [268]:
def symbol_grams_spaces(sents, iterations):
    transformed = []
    for _ in range(iterations):
        if not transformed:
            transformed = sents
        phrases = Phrases(transformed, scoring='npmi', threshold=0, min_count=3)
        transformed = phrases[transformed]
        transformed = split_spaces(transformed)
    return transformed

In [269]:
result_spaces = symbol_grams_spaces(symbol_sents, 3)

С учётом пробелов получается результат, более близкий к желаемому, но появляются дополнительные пробелы. Предположительно, это связано с тем, что артикль склеивается с пробелом при каждой итерации, что вполне объяснимо тем, что для артикля это самая частотная пара.

In [270]:
'|'.join(list(result_spaces)[1]) 

'V_a_l_e_n_t_i_n|e| |M_i_c_h_a_e_l| |S|m_i_t_h| |w_a_s| |a_s| |r_e|a_l| |a_s| |t_a_x_e_s| |b|u_t| |h_e| |w_a_s| |a| | | | | | | |r_a_c|e| |o_f| | |o_n|e'